In [25]:
import sqlalchemy
import json 

with open("./db_config.json", "r") as f:
    config = json.load(f)

USER = config["DB_USER"]
PW = config["DB_PASSWORD"]
HOST = config["DB_HOST"]
PORT = config["DB_PORT"]
DB = config["DB_NAME"]

# DB 연결
engine = sqlalchemy.create_engine(f"mysql+pymysql://{USER}:{PW}@{HOST}:{PORT}/{DB}")

with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT VERSION();"))
    print("Connect Success. Version is", result.fetchone())


Connect Success. Version is ('8.4.4',)


In [26]:
# 경기장 정보를 가져옵니다.
import pandas as pd

strQuery = "select * from stadium"
stadium_df = pd.read_sql(strQuery, engine)

stadium_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   stadium_code  60 non-null     object
 1   sports_type   60 non-null     object
 2   stadium_name  60 non-null     object
 3   region        60 non-null     object
 4   address       60 non-null     object
dtypes: object(5)
memory usage: 2.5+ KB


In [27]:
# 경기 정보를 가져옵니다.
strQuery = "select * from sports_game"
games_df = pd.read_sql(strQuery, engine)
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3631 entries, 0 to 3630
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   game_id        3631 non-null   int64          
 1   stadium_code   3631 non-null   object         
 2   sports_type    3631 non-null   object         
 3   game_date      3631 non-null   object         
 4   day_of_week    3631 non-null   object         
 5   is_holiday     0 non-null      object         
 6   start_time     3631 non-null   timedelta64[ns]
 7   end_time       3631 non-null   timedelta64[ns]
 8   home_team_win  3631 non-null   int64          
 9   match_type     3610 non-null   object         
 10  audience       0 non-null      object         
dtypes: int64(2), object(7), timedelta64[ns](2)
memory usage: 312.2+ KB


In [28]:
# 경기 정보를 가져옵니다.
strQuery = "select * from weather"
weather_df = pd.read_sql(strQuery, engine)
weather_df.head()

,weather_id,stadium_code,region,weather_date,weather_time,temperature,precipitation,wind_speed,wind_dir_deg,humidity,snow_depth,cloud_amount,low_cloud_amt,visibility
0,1,CC01,춘천,2023-01-01,0 days 00:00:00,-2.5,NaN,0.6,340.0,88.0,1.4,9.0,NaN,630.0
1,2,CC01,춘천,2023-01-01,0 days 01:00:00,-2.8,NaN,0.5,360.0,89.0,1.4,0.0,NaN,561.0
2,3,CC01,춘천,2023-01-01,0 days 02:00:00,-3.0,NaN,0.4,0.0,89.0,1.5,0.0,NaN,598.0
3,4,CC01,춘천,2023-01-01,0 days 03:00:00,-3.2,NaN,0.2,0.0,90.0,1.5,0.0,NaN,513.0
4,5,CC01,춘천,2023-01-01,0 days 04:00:00,-4.7,NaN,0.5,360.0,93.0,1.5,0.0,NaN,268.0


In [29]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450811 entries, 0 to 450810
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype          
---  ------         --------------   -----          
 0   weather_id     450811 non-null  int64          
 1   stadium_code   450811 non-null  object         
 2   region         450811 non-null  object         
 3   weather_date   450811 non-null  object         
 4   weather_time   450811 non-null  timedelta64[ns]
 5   temperature    450532 non-null  float64        
 6   precipitation  151195 non-null  float64        
 7   wind_speed     446194 non-null  float64        
 8   wind_dir_deg   447979 non-null  float64        
 9   humidity       417416 non-null  float64        
 10  snow_depth     8036 non-null    float64        
 11  cloud_amount   333289 non-null  float64        
 12  low_cloud_amt  315877 non-null  float64        
 13  visibility     330832 non-null  float64        
dtypes: float64(9), int64(1), object(3), 

In [30]:
# 경기 정보를 가져옵니다.
strQuery = "select * from traffic_accident"
traffic_accident_df = pd.read_sql(strQuery, engine)
print(len(traffic_accident_df))
traffic_accident_df.head()


34032


,accident_id,region,stadium_code,accident_date,accident_count,death_count,injury_count
0,1,강원 강릉시,GN01,2023-01-01,5,0,10
1,2,강원 강릉시,GN01,2023-01-02,2,0,2
2,3,강원 강릉시,GN01,2023-01-03,3,0,4
3,4,강원 강릉시,GN01,2023-01-04,3,0,4
4,5,강원 강릉시,GN01,2023-01-05,1,0,1


In [ ]:
# 야구 관중수 정보
df_kbo_audience = pd.read_csv('./raw_data/야구_kbo_관중수_2023_2024.csv', encoding='utf-8')
stadium_code_map = {
    "사직": "BS02",
    "청주": "CJ01",
    "창원": "CW01",
    "대구": "DG03",
    "대전": "DJ02",
    "광주": "GJ02",
    "문학": "IC02",
    "이천": "IH01",
    "포항": "PH02",
    "고척": "SO04",
    "잠실": "SO05",
    "수원": "SU03",
    "울산": "US03"
}

df_kbo_audience["stadium_code"] = df_kbo_audience["구장"].map(stadium_code_map)
df_kbo_audience['날짜'] = pd.to_datetime(df_kbo_audience['날짜'], format="%Y.%m.%d").dt.date
df_kbo_audience['관중수'] = df_kbo_audience['관중수'].str.replace(',', '').astype(int)
df_kbo_audience.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   연도            1440 non-null   int64 
 1   날짜            1440 non-null   object
 2   요일            1440 non-null   object
 3   홈             1440 non-null   object
 4   방문            1440 non-null   object
 5   구장            1440 non-null   object
 6   관중수           1440 non-null   int64 
 7   stadium_code  1440 non-null   object
dtypes: int64(2), object(6)
memory usage: 90.1+ KB


In [96]:
# 축구 관중수..
df_klg_audience = pd.read_csv('./raw_data/축구_k-league_관중수_2023_2024.csv', encoding='utf-8')
klg_stadium_codemap = {
    "부산 아시아드": "BS01",
    "천안 종합": "CA01",
    "춘천 송암": "CC01",
    "대구iM뱅크PARK": "DG01",
    "대구 스타디움": "DG02",
    "대전 월드컵": "DJ01",
    "김천 종합": "GC01",
    "광주 전용": "GJ01",
    "강릉 종합": "GN01",
    "김포 솔터": "GP01",
    "인천 전용": "IC01",
    "제주 월드컵": "JE01",
    "전주 월드컵": "JJ01",
    "포항 스틸야드": "PH01",
    "서울 월드컵": "SO01",
    "상암 보조": "SO02",
    "목동 종합": "SO03",
    "수원 월드컵": "SU01",
    "수원 종합": "SU02",
    "울산 문수": "US01",
    "울산 종합": "US02"
}



df_klg_audience["stadium_code"] = df_klg_audience["경기장"].map(klg_stadium_codemap)
df_klg_audience['일자'] = pd.to_datetime(df_klg_audience['일자'], format="%Y.%m.%d").dt.date
df_klg_audience['관중수'] = df_klg_audience['관중수'].str.replace(',', '').astype(int)
df_klg_audience['원정팀'] = df_klg_audience['원정팀'].str.replace(',', '').astype(int)
df_klg_audience.head()
#df_klg_audience.info()

#print(df_klg_audience['일자'].apply(type).value_counts())

#df_klg_audience.경기장.unique()

,시즌,대회,대회명,경기번호,일자,홈팀,상대,경기결과,경기장,관중수,원정팀,비고,stadium_code
0,2023,K리그1,하나원큐 K리그1 2023,1,2023-02-25,울산,전북,2:01,울산 문수,28039,2037,주말,US01
1,2023,K리그1,하나원큐 K리그1 2023,2,2023-02-25,서울,인천,2:01,서울 월드컵,22204,3216,주말,SO01
2,2023,K리그1,하나원큐 K리그1 2023,3,2023-02-25,수원,광주,0:01,수원 월드컵,10348,409,주말,SU01
3,2023,K리그1,하나원큐 K리그1 2023,4,2023-02-26,포항,대구,3:02,포항 스틸야드,14089,1152,주말,PH01
4,2023,K리그1,하나원큐 K리그1 2023,5,2023-02-26,제주,수원FC,0:00,제주 월드컵,8362,181,주말,JE01


In [ ]:
# 배구 관중수..
df_vlg_audience = pd.read_csv('./raw_data/배구_vl_관중수_2023_2024.csv', encoding='utf-8')
vlg_stadium_codemap = {
    "안산상록수체육관": "AS01",
    "천안유관순체육관": "CA02",
    "대전충무체육관": "DJ04",
    "김천실내체육관": "GC02",
    "구미박정희체육관": "GM01",
    "페퍼스타디움": "HS01",
    "화성종합실내체육관": "HS02",
    "인천계양체육관": "IC03",
    "인천삼산월드체육관": "IC04",
    "서울장충체육관": "SO08",
    "수원체육관": "SU05",
    "의정부체육관": "UJ01",
    "경민대학교 기념관(체육관)": "UJ02"
}

df_vlg_audience["stadium_code"] = df_vlg_audience["place"].map(vlg_stadium_codemap)
df_vlg_audience["gDate"] = pd.to_datetime(df_vlg_audience["gDate"], format="%Y.%m.%d").dt.date
#print(df_vlg_audience['gDate'].apply(type).value_counts())
df_vlg_audience.head()




,hTeamCode,aTeamCode,gDate,sTime,eTime,hSetPoint,aSetPoint,place,city,spectators,aTeamName,totalTime,fetch_gNum,fetch_gPart_success,stadium_code
0,1001,1004,2022-10-22,14:00,16:03,3,1,인천계양체육관,인천,1987,KB손해보험,1:52,1,201,IC03
1,2001,2002,2022-10-22,14:00,15:29,3,0,수원체육관,수원,2913,한국도로공사,1:21,2,201,SU05
2,1008,1006,2022-10-23,14:25,15:57,0,3,안산상록수체육관,안산,1661,한국전력,1:24,3,201,AS01
3,2006,2005,2022-10-23,16:00,17:31,0,3,화성종합실내체육관,화성,2579,GS칼텍스,1:23,4,201,HS02
4,1002,1005,2022-10-25,19:00,21:05,1,3,대전충무체육관,대전,931,현대캐피탈,1:54,5,201,DJ04


<h3>1차 데이터셋 구성</h3>



In [112]:
# 모든 시간을 XX:00:00 으로 바꾸는 함수.
def round_by_half_hour(td):
    hour = td.components.hours
    minute = td.components.minutes

    if minute < 30:
        return pd.to_timedelta(f"{hour}:00:00")
    else:
        return pd.to_timedelta(f"{hour + 1}:00:00")

In [113]:
# pip install holidays  # 공휴일 정보 가져오은 라이블러리
# 대한민국 공휴일 정보 로딩
import holidays

kr_holidays = holidays.KR()

def is_korean_holiday(date_obj):
    """
    date_obj: datetime.date 타입
    반환값: 1이면 공휴일 또는 주말, 0이면 평일
    """
    # 주말인지 확인 (토요일=5, 일요일=6)
    if date_obj.weekday() in [5, 6]:
        return 1
    # 공휴일인지 확인
    if date_obj in kr_holidays:
        return 1
    return 0

In [114]:
from datetime import date

print(is_korean_holiday(date(2025, 5, 5)))  # True (어린이날)
print(is_korean_holiday(date(2025, 5, 7)))  # True (대체휴일)
print(is_korean_holiday(date(2025, 6, 1)))  # False

1
0
1


In [115]:
df_kbo_audience.head()

,연도,날짜,요일,홈,방문,구장,관중수,stadium_code
0,2023,2023-04-01,토,키움,한화,고척,16000,SO04
1,2023,2023-04-01,토,SSG,KIA,문학,23000,IC02
2,2023,2023-04-01,토,KT,LG,수원,18700,SU03
3,2023,2023-04-01,토,두산,롯데,잠실,23750,SO05
4,2023,2023-04-01,토,삼성,NC,대구,24000,DG03


In [168]:
ds_df = games_df
ds_df.loc[ds_df['stadium_code'] == 'DJ03', 'stadium_code'] = 'DJ02'


ds_df = games_df.merge(df_kbo_audience[['stadium_code','날짜', '관중수']], 
                                                left_on=['stadium_code', 'game_date'],
                                                right_on=['stadium_code', '날짜'], 
                                                how='left')

#print(df_kbo_audience['날짜'].apply(type).value_counts())
#print(ds_df['game_date'].apply(type).value_counts())
ds_df = ds_df.drop(columns=["날짜"])
ds_df.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   game_id        3677 non-null   int64          
 1   stadium_code   3677 non-null   object         
 2   sports_type    3677 non-null   object         
 3   game_date      3677 non-null   object         
 4   day_of_week    3677 non-null   object         
 5   is_holiday     0 non-null      object         
 6   start_time     3677 non-null   timedelta64[ns]
 7   end_time       3677 non-null   timedelta64[ns]
 8   home_team_win  3677 non-null   int64          
 9   match_type     3656 non-null   object         
 10  audience       0 non-null      object         
 11  관중수            1609 non-null   float64        
dtypes: float64(1), int64(2), object(7), timedelta64[ns](2)
memory usage: 344.8+ KB


In [169]:
# 축구경기 관중수 추가..
ds_df = ds_df.merge(df_klg_audience[['stadium_code','일자', '관중수']], 
                                                left_on=['stadium_code', 'game_date'],
                                                right_on=['stadium_code', '일자'], 
                                                how='left')
ds_df = ds_df.drop(columns=["일자"])
ds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   game_id        3677 non-null   int64          
 1   stadium_code   3677 non-null   object         
 2   sports_type    3677 non-null   object         
 3   game_date      3677 non-null   object         
 4   day_of_week    3677 non-null   object         
 5   is_holiday     0 non-null      object         
 6   start_time     3677 non-null   timedelta64[ns]
 7   end_time       3677 non-null   timedelta64[ns]
 8   home_team_win  3677 non-null   int64          
 9   match_type     3656 non-null   object         
 10  audience       0 non-null      object         
 11  관중수_x          1609 non-null   float64        
 12  관중수_y          456 non-null    float64        
dtypes: float64(2), int64(2), object(7), timedelta64[ns](2)
memory usage: 373.6+ KB


In [170]:
# 배구 경기 관중수 추가..
ds_df = ds_df.merge(df_vlg_audience[['stadium_code','gDate', 'spectators']], 
                                                left_on=['stadium_code', 'game_date'],
                                                right_on=['stadium_code', 'gDate'], 
                                                how='left')
ds_df = ds_df.drop(columns=["gDate"])
#ds_df.info()

In [171]:
ds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   game_id        3677 non-null   int64          
 1   stadium_code   3677 non-null   object         
 2   sports_type    3677 non-null   object         
 3   game_date      3677 non-null   object         
 4   day_of_week    3677 non-null   object         
 5   is_holiday     0 non-null      object         
 6   start_time     3677 non-null   timedelta64[ns]
 7   end_time       3677 non-null   timedelta64[ns]
 8   home_team_win  3677 non-null   int64          
 9   match_type     3656 non-null   object         
 10  audience       0 non-null      object         
 11  관중수_x          1609 non-null   float64        
 12  관중수_y          456 non-null    float64        
 13  spectators     534 non-null    float64        
dtypes: float64(3), int64(2), object(7), timedelta64[ns](2)
m

In [172]:
ds_df.loc[ds_df['sports_type'] == '야구', 'audience'] = ds_df['관중수_x']
ds_df.loc[ds_df['sports_type'] == '축구', 'audience'] = ds_df['관중수_y']
ds_df.loc[ds_df['sports_type'] == '배구', 'audience'] = ds_df['spectators']

ds_df = ds_df.drop(columns=['관중수_x', '관중수_y', 'spectators'])
ds_df.head()

,game_id,stadium_code,sports_type,game_date,day_of_week,is_holiday,start_time,end_time,home_team_win,match_type,audience
0,1,DJ02,야구,2023-03-13,월,None,0 days 14:00:00,0 days 17:00:00,1,시범경기,NaN
1,2,SO04,야구,2023-03-13,월,None,0 days 14:00:00,0 days 17:00:00,0,시범경기,NaN
2,3,CW01,야구,2023-03-13,월,None,0 days 14:00:00,0 days 17:00:00,0,시범경기,NaN
3,4,BS02,야구,2023-03-13,월,None,0 days 14:00:00,0 days 17:00:00,0,시범경기,NaN
4,5,DG03,야구,2023-03-13,월,None,0 days 14:00:00,0 days 17:00:00,1,시범경기,NaN


In [173]:
# 1차 데이터셋 구성.

# 경기장 → 지역 매핑
# ds1_df = games_df.merge(stadium_df[['stadium_code','region']], on='stadium_code', how='left')
ds1_df = ds_df.merge(stadium_df[['stadium_code','region']], on='stadium_code', how='left')

# 닐짜, 지역 기준 사고건수. 부상자수, 사망자수 를 가져온다. 
ds1_df = ds1_df.merge(traffic_accident_df[['region','accident_date', 'accident_count', 'injury_count', 'death_count']], 
                                                left_on=['region', 'game_date'],
                                                right_on=['region', 'accident_date'], 
                                                how='left')

# 필요없는 컬럼 삭제..
ds1_df = ds1_df.drop(columns=['accident_date', 'end_time'])

# 시작 시간으로부터 기장정보를 가져오기 위한 비교시간을 계산한다.
ds1_df['start_time_cmp'] = ds1_df['start_time'].apply(round_by_half_hour)

# 날씨정보를 가져온다.
# 먼저 full merge 수행 (조건을 일단 무시하고 전체 cross join에 가까운 병합 수행)
ds1_df = ds1_df.merge(
    weather_df[['weather_date', 'weather_time', 'region', 'temperature', 'precipitation', 'snow_depth']],
    left_on=['game_date', 'start_time_cmp'],
    right_on=['weather_date', 'weather_time'],
    how='left',
    suffixes=('', '_weather')
)

# region_weather(문자열)이 region(문자열)에 포함 여부로 필터링
ds1_df = ds1_df[
    ds1_df.apply(lambda row: row['region_weather'] in row['region'], axis=1)
]

# 필요없는 칼럼 삭제
ds1_df = ds1_df.drop(columns=['weather_date', 'weather_time', 'region_weather', 'start_time_cmp'])

# holiday 값 적용
gameDate = pd.to_datetime(ds1_df['game_date'])
ds1_df['is_holiday'] = gameDate.dt.date.apply(is_korean_holiday).astype(int)

print(ds1_df.is_holiday.unique())

# NaN 데이타 0으로 치환
# ds1_df = ds1_df.fillna(0)

# 상위 데이타 몇 개 출력
print(ds1_df.head())

# csv 로 1차 데이타셋 저장
ds1_df.to_csv("1st-dataset.csv", index=False, encoding='utf-8')

[0 1]
     game_id stadium_code sports_type   game_date day_of_week  is_holiday  \
7          1         DJ02          야구  2023-03-13           월           0   
28         2         SO04          야구  2023-03-13           월           0   
63         3         CW01          야구  2023-03-13           월           0   
91         4         BS02          야구  2023-03-13           월           0   
112        5         DG03          야구  2023-03-13           월           0   

         start_time  home_team_win match_type audience  region  \
7   0 days 14:00:00              1       시범경기      NaN   대전 중구   
28  0 days 14:00:00              0       시범경기      NaN  서울 구로구   
63  0 days 14:00:00              0       시범경기      NaN  경남 창원시   
91  0 days 14:00:00              0       시범경기      NaN  부산 동래구   
112 0 days 14:00:00              1       시범경기      NaN  대구 수성구   

     accident_count  injury_count  death_count  temperature  precipitation  \
7               3.0           4.0          0.0          

In [24]:
ds1_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3577 entries, 7 to 93416
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   game_id         3577 non-null   int64          
 1   stadium_code    3577 non-null   object         
 2   sports_type     3577 non-null   object         
 3   game_date       3577 non-null   object         
 4   day_of_week     3577 non-null   object         
 5   is_holiday      3577 non-null   int64          
 6   start_time      3577 non-null   timedelta64[ns]
 7   home_team_win   3577 non-null   int64          
 8   match_type      3577 non-null   object         
 9   audience        3577 non-null   int64          
 10  region          3577 non-null   object         
 11  accident_count  3577 non-null   float64        
 12  injury_count    3577 non-null   float64        
 13  death_count     3577 non-null   float64        
 14  temperature     3577 non-null   float64     